In [4]:
import os
os.path.expanduser('~/.keras/models')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
model_conv = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [6]:
model_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [7]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator = data_generator.flow_from_directory(
    'PetImages',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


In [8]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

Found 1400 images belonging to 2 classes.


In [9]:
from tensorflow.keras import models, layers, optimizers

In [19]:
model_conv.trainable = False

In [21]:
model = models.Sequential()
model.add(model_conv)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.fit_generator(generator, steps_per_epoch=100, epochs=30, validation_data=generator_test, validation_steps=30)

Epoch 1/30
 19/100 [====>.........................] - ETA: 2:51 - loss: 0.6925 - accuracy: 0.5158